In [ ]:
LINK_WITH_COMET=False

In [ ]:
!pip install transformers -q
!pip install ftfy -q

if LINK_WITH_COMET:
    !pip install comet_ml -q

In [ ]:
import pickle
#from google.colab import drive

import pandas as pd
import numpy as np

import torch

from scipy import stats

from datetime import datetime

import ftfy

from transformers import (get_linear_schedule_with_warmup, 
                          get_cosine_with_hard_restarts_schedule_with_warmup,
                          AutoTokenizer, 
                          AutoModel,
                          AutoModelForSequenceClassification, 
                          BatchEncoding
)

import os

from tqdm.auto import tqdm

import json

if LINK_WITH_COMET:
    from comet_ml import Experiment

In [ ]:
WORKING_FOLDER="."
TRAIN_OUTPUT_FOLDER="./trained_model"


API_KEYS_FILE="/content/drive/MyDrive/unicamp/ia368v_dd/api_keys_20230324.json"

MS_MARCO_SPLIT="ms_marco_tiny_data_split.pkl"
MS_MARCO_TINY_URL="https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv"

In [ ]:
MODEL_NAME='microsoft/MiniLM-L12-H384-uncased'

NUMBER_OF_EVALUATION_SAMPLES=1000

EPSILON=1e-8

In [ ]:
#drive.mount('/content/drive', force_remount=True)

In [ ]:
os.chdir(WORKING_FOLDER)

In [ ]:
if LINK_WITH_COMET:
    with open(API_KEYS_FILE) as inputFile:
        api_keys = json.load(inputFile)

    experiment = Experiment(api_key=api_keys['comet_ml'], 
                            project_name="Dense Passage Retriever",
                            workspace="eduseiti")

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

## Read the MS MARCO data split, if available

In [ ]:
if os.path.exists(MS_MARCO_SPLIT):
    with open(MS_MARCO_SPLIT, "rb") as inputFile:
        ms_marco_data = pickle.load(inputFile)

    train_df = ms_marco_data['train']
    validation_df = ms_marco_data['validation']
else:
    print("Need to import and fix the training dataset...")

## Import and fix training dataset

In [ ]:
if not 'train_df' in locals():
    if not os.path.exists(os.path.basename(MS_MARCO_TINY_URL)):
        #!wget {MS_MARCO_TINY_URL}
         print("Data split has already been loaded...")
    else:
        print("Training dataset already downloaded...")

    ms_df = pd.read_csv(os.path.basename(MS_MARCO_TINY_URL), sep="\t", header=None, names=['topic', 'positive', 'negative'])
    pd.set_option('display.max_colwidth', None)

    display(ms_df.head())

    ms_df['positive'] = ms_df['positive'].apply(lambda text: ftfy.fix_text(text))
    # ms_df = ms_df.drop('negative', axis=1)

else:
    print("Data split has already been loaded...")

Split evaluation part

In [ ]:
if not 'train_df' in locals():
    print("ms_df.shape={}".format(ms_df.shape))

    split_entries = np.random.choice(list(range(ms_df.shape[0])), NUMBER_OF_EVALUATION_SAMPLES, replace=False)

    train_df = ms_df.iloc[np.setdiff1d(list(range(ms_df.shape[0])), split_entries)].reset_index(drop=True)
    validation_df = ms_df.iloc[split_entries].reset_index(drop=True)
    
    print("train_df.shape={}".format(train_df.shape))
    print("validation_df.shape={}".format(validation_df.shape))
    
    with open(MS_MARCO_SPLIT, "wb") as outputFile:
        pickle.dump({'train': train_df, 
                    'validation': validation_df}, outputFile, pickle.HIGHEST_PROTOCOL)
        
else:
    print("Data split has already been loaded...")

## Create dataset class

In [ ]:
class DensePassageRetrieverDataset(torch.utils.data.Dataset):

    def __init__(self, ms_df, tokenizer):

        self.tokenized_topics = tokenizer(ms_df['topic'].tolist(), return_length=True)
        self.tokenized_passage = tokenizer(ms_df['positive'].tolist(), return_length=True)

        print("Topics tokens size stats:\n{}\n".format(stats.describe(self.tokenized_topics['length'])))
        print("Passages tokens size stats:\n{}\n".format(stats.describe(self.tokenized_passage['length'])))

        self.shuffle()


    def shuffle(self):
        self.samples_order = list(range(len(self.tokenized_topics['input_ids'])))
        np.random.shuffle(self.samples_order)


    def __len__(self):
        return len(self.tokenized_topics['input_ids'])


    def __getitem__(self, index):

        return {'passages': {'input_ids': self.tokenized_passage['input_ids'][self.samples_order[index]],
                            'attention_mask': self.tokenized_passage['attention_mask'][self.samples_order[index]]},
                'topics' : {'input_ids': self.tokenized_topics['input_ids'][self.samples_order[index]],
                            'attention_mask': self.tokenized_topics['attention_mask'][self.samples_order[index]]}}

In [ ]:
class DPRCollator(object):
    def __init__(self, type='passages', tokenizer=None):
        self.type = type
        self.tokenizer = tokenizer


    def __call__(self, batch):

        padded_batch = self.tokenizer.pad([item[self.type] for item in batch], return_tensors='pt')

        return BatchEncoding(padded_batch)

## Create the evaluation function

In [ ]:
def compute_loss(passages_outputs, topics_outputs):

    # print("passages_outputs.keys()={}".format(passages_outputs.keys()))
    # print("topics_outputs.keys()={}".format(topics_outputs.keys()))

    # print("passages_outputs.pooler_output.shape={}".format(passages_outputs.pooler_output.shape))
    # print("passages_outputs.last_hidden_state.shape={}".format(passages_outputs.last_hidden_state.shape))

    passages_cls = passages_outputs.last_hidden_state[:, 0, :]
    topics_cls = topics_outputs.last_hidden_state[:, 0, :]

    # print("passages_cls.shape={}, type(passages_cls)={}".format(passages_cls.shape, type(passages_cls)))


    all_passages_all_topics_dot_product = torch.mm(passages_cls, topics_cls.t())
    passages_and_positive_topics = all_passages_all_topics_dot_product.diag().unsqueeze(1)

    dot_product_differences = all_passages_all_topics_dot_product - passages_and_positive_topics

    loss = torch.log(torch.sum(torch.exp(dot_product_differences), dim=1))

    #
    # The code below is to confirm the loss above is the same if computed exactly as state in 
    # the DPR paper.
    #
    # dot_prod_exp = torch.exp(all_passages_all_topics_dot_product).fill_diagonal_(0.0)
    # print(dot_prod_exp.shape)
    # print(passages_and_positive_topics.shape)
    

    # all_passages_and_negative_topics_sum = torch.sum(dot_prod_exp, dim=1)
    # print(all_passages_and_negative_topics_sum.shape)

    # passage_positive_exp = torch.exp(passages_and_positive_topics.squeeze(1))

    # verification_loss = -torch.log(passage_positive_exp / (passage_positive_exp + all_passages_and_negative_topics_sum))

    # print(loss)
    # print(verification_loss)
    # print(verification_loss.shape)

    # print(loss==verification_loss)



    non_zeroed_losses = (loss > EPSILON).float().sum()

    if non_zeroed_losses > 0.0:
        final_loss = torch.sum(loss) / non_zeroed_losses
    else:
        final_loss = torch.mean(loss)

    return final_loss

In [ ]:
def evaluate(device,
             passages_model, 
             topics_model, 
             passages_dataloader, 
             topics_dataloader, 
             min_eval_loss,
             current_epoch=0):
    
    eval_losses = []

    passages_model.eval()
    topics_model.eval()

    with torch.no_grad():
        for batch in tqdm(list(zip(passages_dataloader, topics_dataloader)), mininterval=0.5, desc="Eval", disable=False):
            
            passages_outputs = passages_model(**batch[0].to(device))
            topics_outputs = topics_model(**batch[1].to(device))

            eval_losses.append(compute_loss(passages_outputs, topics_outputs).cpu().numpy())

    final_loss = np.mean(eval_losses)

    print("Eval loss: {:0.3f}".format(final_loss))

    if LINK_WITH_COMET:
        experiment.log_metrics({'eval_loss': final_loss},
                               epoch=current_epoch)


    if min_eval_loss['loss'] > final_loss:
        print("New minimal validation loss; saving model...")

        training_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        checkpoint_name = "checkpoint_{}_{:.4f}".format(training_timestamp, final_loss)
        passages_model.save_pretrained(os.path.join(TRAIN_OUTPUT_FOLDER, checkpoint_name, "_passages"))
        topics_model.save_pretrained(os.path.join(TRAIN_OUTPUT_FOLDER, checkpoint_name, "_topics"))

        min_eval_loss['checkpoint_name'] = checkpoint_name
        min_eval_loss['loss'] = final_loss

## Instantiate the tokenizer, models (passage and topic), dataset and dataloaders

### Define the model/training hyperparameters

In [ ]:

hyperparameters = {
    'batch_size': 32
}

### Instantiate the model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
passages_model = AutoModel.from_pretrained(MODEL_NAME).to(device)
topics_model = AutoModel.from_pretrained(MODEL_NAME).to(device)

### Instantiate the datasets

In [ ]:
train_dataset = DensePassageRetrieverDataset(train_df, tokenizer)

In [ ]:
eval_dataset = DensePassageRetrieverDataset(validation_df, tokenizer)

### Now create the dataloaders, 2 for each split (train, eval) to supply the passages and the topics data

In [ ]:
train_passages_dataloader = torch.utils.data.DataLoader(train_dataset, 
                                                        batch_size=hyperparameters['batch_size'], 
                                                        shuffle=False, 
                                                        collate_fn=DPRCollator('passages', tokenizer))

train_topics_dataloader = torch.utils.data.DataLoader(train_dataset, 
                                                      batch_size=hyperparameters['batch_size'], 
                                                      shuffle=False, 
                                                      collate_fn=DPRCollator('topics', tokenizer))

eval_passages_dataloader = torch.utils.data.DataLoader(eval_dataset, 
                                                       batch_size=hyperparameters['batch_size'], 
                                                       shuffle=False, 
                                                       collate_fn=DPRCollator('passages', tokenizer))

eval_topics_dataloader = torch.utils.data.DataLoader(eval_dataset, 
                                                     batch_size=hyperparameters['batch_size'], 
                                                     shuffle=False, 
                                                     collate_fn=DPRCollator('topics', tokenizer))

## Finally, start trainining

In [ ]:
min_eval_loss = {"loss": 1000,
                 "checkpoint_name": None}

In [ ]:
hyperparameters['epochs'] = 10
hyperparameters['num_training_steps'] = hyperparameters['epochs'] * int(len(train_dataset) // hyperparameters['batch_size'])
hyperparameters['num_warmup_steps'] = 0
hyperparameters['learning_rate'] = 1e-4
hyperparameters['num_cosine_scheduler_cycles'] = 1

In [ ]:
passages_optimizer = torch.optim.AdamW(passages_model.parameters(), lr=hyperparameters['learning_rate'])
topics_optimizer = torch.optim.AdamW(topics_model.parameters(), lr=hyperparameters['learning_rate'])

In [ ]:
passages_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(passages_optimizer, 
                                                                        hyperparameters['num_warmup_steps'], 
                                                                        hyperparameters['num_training_steps'],
                                                                        num_cycles=hyperparameters['num_cosine_scheduler_cycles'])

In [ ]:
topics_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(topics_optimizer, 
                                                                      hyperparameters['num_warmup_steps'], 
                                                                      hyperparameters['num_training_steps'],
                                                                      num_cycles=hyperparameters['num_cosine_scheduler_cycles'])

In [ ]:
if LINK_WITH_COMET:
    experiment.log_parameters(hyperparameters)

In [ ]:
evaluate(device,
         passages_model=passages_model, 
         topics_model=topics_model,
         passages_dataloader=eval_passages_dataloader,
         topics_dataloader=eval_topics_dataloader,
         min_eval_loss=min_eval_loss,
         current_epoch=-1)

In [ ]:
current_training_step = 0

for epoch in tqdm(range(hyperparameters['epochs']), desc='Epochs'):
    
    passages_model.train()
    topics_model.train()

    train_losses = []
    
    tqdm_batches = tqdm(list(zip(train_passages_dataloader, train_topics_dataloader)), mininterval=0.5, desc='Train', disable=False)

    for batch in tqdm_batches:

        current_training_step += 1

        passages_optimizer.zero_grad()
        topics_optimizer.zero_grad()

        passages_outputs = passages_model(**batch[0].to(device))
        topics_outputs = topics_model(**batch[1].to(device))

        final_loss = compute_loss(passages_outputs, topics_outputs)

        final_loss.backward()

        passages_optimizer.step()
        topics_optimizer.step()

        passages_scheduler.step()
        topics_scheduler.step()

        train_losses.append(final_loss.detach().cpu().numpy())

        tqdm_batches.set_description("Loss {:0.4f}".format(train_losses[-1]))

        if LINK_WITH_COMET:
            experiment.log_metrics({'train loss': train_losses[-1],
                                    'learning_rate': passages_scheduler.get_last_lr()},
                                    step=current_training_step)

    print("Epoch: {}, Training loss: {:0.4f}".format(epoch + 1, np.mean(train_losses)))
    
    if LINK_WITH_COMET:
        experiment.log_metrics({'train loss': np.mean(train_losses)},
                               epoch=epoch)


    evaluate(device,
             passages_model=passages_model, 
             topics_model=topics_model,
             passages_dataloader=eval_passages_dataloader,
             topics_dataloader=eval_topics_dataloader,
             min_eval_loss=min_eval_loss,
             current_epoch=epoch)
    
    train_dataset.shuffle()

In [ ]:
experiment.end()